In [132]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.layers import Reshape
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend

import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import os
from tensorflow.keras.utils import to_categorical

In [133]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

from tensorflow import keras
from tensorflow.keras.layers import Bidirectional
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Input,Dense,Flatten,Dropout,Conv1D,MaxPooling1D,concatenate
from keras.utils import to_categorical

In [134]:
features = pd.read_csv("C:/pamap2.csv", index_col=0)

In [135]:
label = LabelEncoder()
features['activityID'] = label.fit_transform(features['activityID'])
features.head()

,timestamp,activityID,heartrate,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,...,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,ankleOrientation1,ankleOrientation2,ankleOrientation3,ankleOrientation4,subject_id
2932,37.70,0,100.0,30.375,2.30106,7.25857,6.09259,2.20720,7.24042,5.95555,...,-0.048878,-0.006328,-60.2040,-37.1225,-57.8847,1.0,0.0,0.0,0.0,1
2943,37.81,0,100.0,30.375,2.24615,7.48180,5.55219,2.25130,7.40595,5.65313,...,-0.017912,0.015243,-60.7820,-36.2116,-58.4891,1.0,0.0,0.0,0.0,1
2954,37.92,0,100.0,30.375,2.30000,7.10681,6.09309,2.34613,7.46611,6.12103,...,-0.011798,0.011387,-61.4531,-36.2983,-58.4845,1.0,0.0,0.0,0.0,1
2965,38.03,0,100.0,30.375,2.49455,7.52335,6.17157,2.59106,7.76653,6.27112,...,-0.009233,0.005823,-62.0065,-36.7217,-58.3608,1.0,0.0,0.0,0.0,1
2976,38.14,0,101.0,30.375,2.71654,8.30596,4.78671,2.59596,7.94641,4.86635,...,-0.042562,-0.009289,-60.4311,-37.5412,-59.0013,1.0,0.0,0.0,0.0,1


In [136]:
features['activityID'].value_counts()

3    20998
0    17575
2    17256
1    16881
5    16857
6    10699
7     9585
4     8665
Name: activityID, dtype: int64

In [137]:
indexes = features[features['activityID'] == 0].index
features.drop(indexes , inplace=True)

In [138]:

X = features[['handAcc16_1' , 'handAcc16_2' , 'handAcc16_3' ,
    'handAcc6_1' , 'handAcc6_2' , 'handAcc6_3' ,'chestAcc16_1', 'chestAcc16_2',  'chestAcc16_3',   
    'chestAcc6_1',  'chestAcc6_2',  'chestAcc6_3']]

y= features['activityID']

scaler = StandardScaler()

X = scaler.fit_transform(X)

scaled_X = pd.DataFrame(data = X, columns = ['handAcc16_1' , 'handAcc16_2' , 'handAcc16_3' ,
    'handAcc6_1' , 'handAcc6_2' , 'handAcc6_3' ,  'chestAcc16_1', 'chestAcc16_2',  'chestAcc16_3',   
    'chestAcc6_1',  'chestAcc6_2',  'chestAcc6_3'
    ])
scaled_X['label'] = y.values

In [139]:


X_train, X_test, y_train, y_test = train_test_split(scaled_X, 
                                                    scaled_X["label"],
                                                    test_size = 0.2, 
                                                    shuffle = True, 
                                                    random_state = 21)

In [140]:
    
TIME_STEPS = 40
STEP = 10

In [141]:


def create_dataset(X, y, time_steps, step):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [142]:
X_train, y_train = create_dataset(X_train[['handAcc16_1' , 'handAcc16_2' , 'handAcc16_3' ,
    'handAcc6_1' , 'handAcc6_2' , 'handAcc6_3' ]], X_train.label, TIME_STEPS,
                                  STEP)
X_test, y_test = create_dataset(X_test[['handAcc16_1' , 'handAcc16_2' , 'handAcc16_3' ,
    'handAcc6_1' , 'handAcc6_2' , 'handAcc6_3']], X_test.label, TIME_STEPS, STEP)



C:\Users\qtx01\AppData\Local\Temp\ipykernel_28264\302130513.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(stats.mode(labels)[0][0])
C:\Users\qtx01\AppData\Local\Temp\ipykernel_28264\302130513.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.a

In [143]:
enc = OneHotEncoder(handle_unknown = "ignore", sparse = False)
enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

C:\Users\qtx01\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [144]:
print("X_train.shape: ", X_train.shape)
print("X_test.shape ", X_test.shape)
print("y_train.shape ", y_train.shape)
print("y_test.shape ", y_test.shape)


X_train.shape:  (8072, 40, 6)
X_test.shape  (2015, 40, 6)
y_train.shape  (8072, 7)
y_test.shape  (2015, 7)


In [145]:
#Epoch 500

verbose,epochs,batch_size=1,30,64 

n_timesteps,n_features,n_outputs=X_train.shape[1],X_train.shape[2],y_train.shape[1]

In [146]:
n_timesteps, n_features, n_outputs

(40, 6, 7)

In [147]:

model = Sequential()
model.add(LSTM(64,input_shape = [n_timesteps, n_features], return_sequences=True))
model.add(LSTM(128))
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(n_outputs, activation = "softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 40, 64)            18176     
                                                                 
 lstm_5 (LSTM)               (None, 128)               98816     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 7)                 455       
                                                                 
Total params: 125703 (491.03 KB)
Trainable params: 125

In [148]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [149]:
history = model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_split = 0.1, shuffle = True)
print(model.summary())

Epoch 1/10
227/227 [==============================] - 32s 101ms/step - loss: 1.5096 - accuracy: 0.3918 - val_loss: 1.2817 - val_accuracy: 0.4876
Epoch 2/10
227/227 [==============================] - 21s 91ms/step - loss: 1.4069 - accuracy: 0.4339 - val_loss: 1.2950 - val_accuracy: 0.4629
Epoch 3/10
227/227 [==============================] - 21s 93ms/step - loss: 1.3930 - accuracy: 0.4491 - val_loss: 1.2825 - val_accuracy: 0.4926
Epoch 4/10
227/227 [==============================] - 20s 90ms/step - loss: 1.3705 - accuracy: 0.4535 - val_loss: 1.2744 - val_accuracy: 0.4864
Epoch 5/10
227/227 [==============================] - 20s 90ms/step - loss: 1.3526 - accuracy: 0.4537 - val_loss: 1.2793 - val_accuracy: 0.4752
Epoch 6/10
227/227 [==============================] - 20s 89ms/step - loss: 1.3428 - accuracy: 0.4672 - val_loss: 1.2681 - val_accuracy: 0.4765
Epoch 7/10
227/227 [==============================] - 20s 90ms/step - loss: 1.3335 - accuracy: 0.4671 - val_loss: 1.2445 - val_accuracy

In [150]:


base_loss,base_accuracy=model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)
modelpth='pamap2.pth'
tf.keras.models.save_model(model, modelpth, include_optimizer=False) 
score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))


32/32 [==============================] - 2s 59ms/step - loss: 1.3219 - accuracy: 0.4819
INFO:tensorflow:Assets written to: pamap2.pth\assets


INFO:tensorflow:Assets written to: pamap2.pth\assets


Accuracy >48.188585
Base Loss >1.32


HAND - HAND

In [151]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X, 
                                                    scaled_X["label"],
                                                    test_size = 0.95, 
                                                    shuffle = True, 
                                                    random_state = 21)

In [152]:
X_train, y_train = create_dataset(X_train[['handAcc16_1' , 'handAcc16_2' , 'handAcc16_3' ,
    'handAcc6_1' , 'handAcc6_2' , 'handAcc6_3' ]], X_train.label, TIME_STEPS,
                                  STEP)
X_test, y_test = create_dataset(X_test[[
    'chestAcc16_1', 'chestAcc16_2',  'chestAcc16_3',   
    'chestAcc6_1',  'chestAcc6_2',  'chestAcc6_3']], X_test.label, TIME_STEPS, STEP)

print(X_train.shape, y_train.shape)

C:\Users\qtx01\AppData\Local\Temp\ipykernel_28264\302130513.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(stats.mode(labels)[0][0])
C:\Users\qtx01\AppData\Local\Temp\ipykernel_28264\302130513.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.a

(501, 40, 6) (501, 1)


In [153]:
enc = OneHotEncoder(handle_unknown = "ignore", sparse = False)
enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

C:\Users\qtx01\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [154]:
print("X_train.shape: ", X_train.shape)
print("X_test.shape ", X_test.shape)
print("y_train.shape ", y_train.shape)
print("y_test.shape ", y_test.shape)

X_train.shape:  (501, 40, 6)
X_test.shape  (9586, 40, 6)
y_train.shape  (501, 7)
y_test.shape  (9586, 7)


In [155]:
base_model = keras.models.load_model(modelpth)

In [156]:
print("Number of layers in the base model: ", len(base_model.layers))
fine_tune_at = 3

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

new_model = Sequential(base_model.layers[:-1])
new_model.add(Dense(4, activation="sigmoid"))
new_model.add(Dense(n_outputs, activation="sigmoid"))
new_model.compile(loss="categorical_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])
new_model.summary()

Number of layers in the base model:  6


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 40, 64)            18176     
                                                                 
 lstm_5 (LSTM)               (None, 128)               98816     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 4)                 260       
                                                                 
 dense_7 (Dense)             (None, 7)                

In [157]:
display_layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
pd.DataFrame(display_layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.src.layers.rnn.lstm.LSTM object at 0x00...,lstm_4,False
1,<keras.src.layers.rnn.lstm.LSTM object at 0x00...,lstm_5,False
2,<keras.src.layers.regularization.dropout.Dropo...,dropout_4,False
3,<keras.src.layers.core.dense.Dense object at 0...,dense_4,True
4,<keras.src.layers.regularization.dropout.Dropo...,dropout_5,True
5,<keras.src.layers.core.dense.Dense object at 0...,dense_5,True


In [158]:
history1 = new_model.fit(X_train, y_train, epochs=10, batch_size = 32, validation_split = 0.1, shuffle = True)
new_model.summary()

Epoch 1/10
15/15 [==============================] - 11s 215ms/step - loss: 2.0259 - accuracy: 0.0400 - val_loss: 1.8728 - val_accuracy: 0.0588
Epoch 2/10
15/15 [==============================] - 1s 48ms/step - loss: 1.9760 - accuracy: 0.0533 - val_loss: 1.8405 - val_accuracy: 0.1569
Epoch 3/10
15/15 [==============================] - 1s 49ms/step - loss: 1.9205 - accuracy: 0.1756 - val_loss: 1.8106 - val_accuracy: 0.3333
Epoch 4/10
15/15 [==============================] - 1s 49ms/step - loss: 1.8804 - accuracy: 0.3289 - val_loss: 1.7827 - val_accuracy: 0.3333
Epoch 5/10
15/15 [==============================] - 1s 48ms/step - loss: 1.8366 - accuracy: 0.4044 - val_loss: 1.7585 - val_accuracy: 0.3529
Epoch 6/10
15/15 [==============================] - 1s 47ms/step - loss: 1.7936 - accuracy: 0.4667 - val_loss: 1.7370 - val_accuracy: 0.3529
Epoch 7/10
15/15 [==============================] - 1s 48ms/step - loss: 1.7629 - accuracy: 0.4667 - val_loss: 1.7180 - val_accuracy: 0.3529
Epoch 8/10


In [159]:
base_model.trainable = True
new_model.compile(loss="categorical_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [160]:

display_layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
pd.DataFrame(display_layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.src.layers.rnn.lstm.LSTM object at 0x00...,lstm_4,True
1,<keras.src.layers.rnn.lstm.LSTM object at 0x00...,lstm_5,True
2,<keras.src.layers.regularization.dropout.Dropo...,dropout_4,True
3,<keras.src.layers.core.dense.Dense object at 0...,dense_4,True
4,<keras.src.layers.regularization.dropout.Dropo...,dropout_5,True
5,<keras.src.layers.core.dense.Dense object at 0...,dense_5,True


In [161]:
history2 = new_model.fit(X_train, y_train, epochs=10, batch_size = 32, validation_split = 0.1, shuffle = True)
new_model.summary()

Epoch 1/10
15/15 [==============================] - 15s 273ms/step - loss: 1.6405 - accuracy: 0.4711 - val_loss: 1.6637 - val_accuracy: 0.3529
Epoch 2/10
15/15 [==============================] - 2s 109ms/step - loss: 1.6228 - accuracy: 0.4711 - val_loss: 1.6525 - val_accuracy: 0.3529
Epoch 3/10
15/15 [==============================] - 2s 101ms/step - loss: 1.5994 - accuracy: 0.4711 - val_loss: 1.6437 - val_accuracy: 0.3529
Epoch 4/10
15/15 [==============================] - 2s 115ms/step - loss: 1.5772 - accuracy: 0.4711 - val_loss: 1.6301 - val_accuracy: 0.3529
Epoch 5/10
15/15 [==============================] - 1s 100ms/step - loss: 1.5589 - accuracy: 0.4711 - val_loss: 1.6247 - val_accuracy: 0.3529
Epoch 6/10
15/15 [==============================] - 2s 104ms/step - loss: 1.5467 - accuracy: 0.4711 - val_loss: 1.6151 - val_accuracy: 0.3529
Epoch 7/10
15/15 [==============================] - 2s 100ms/step - loss: 1.5248 - accuracy: 0.4711 - val_loss: 1.6147 - val_accuracy: 0.3529
Epoch

In [162]:
base_loss,base_accuracy=new_model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)
  
score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))

150/150 [==============================] - 9s 61ms/step - loss: 1.6327 - accuracy: 0.3843
Accuracy >38.431045
Base Loss >1.63


HAND - CHEST

In [163]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X, 
                                                    scaled_X["label"],
                                                    test_size = 0.95, 
                                                    shuffle = True, 
                                                    random_state = 21)

In [164]:
X_train, y_train = create_dataset(X_train[['chestAcc16_1', 'chestAcc16_2',  'chestAcc16_3',   
    'chestAcc6_1',  'chestAcc6_2',  'chestAcc6_3' ]], X_train.label, TIME_STEPS,
                                  STEP)
X_test, y_test = create_dataset(X_test[[
    'chestAcc16_1', 'chestAcc16_2',  'chestAcc16_3',   
    'chestAcc6_1',  'chestAcc6_2',  'chestAcc6_3']], X_test.label, TIME_STEPS, STEP)

print(X_train.shape, y_train.shape)

C:\Users\qtx01\AppData\Local\Temp\ipykernel_28264\302130513.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(stats.mode(labels)[0][0])
C:\Users\qtx01\AppData\Local\Temp\ipykernel_28264\302130513.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.a

(501, 40, 6) (501, 1)


In [165]:
enc = OneHotEncoder(handle_unknown = "ignore", sparse = False)
enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

C:\Users\qtx01\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [166]:
#Epoch 500

verbose,epochs,batch_size=1,10,32 

n_timesteps,n_features,n_outputs=X_train.shape[1],X_train.shape[2],y_train.shape[1]
n_timesteps, n_features, n_outputs

(40, 6, 7)

In [167]:

#CNN MODEL
model2 = Sequential()

model2.add(LSTM(64,input_shape = [n_timesteps, n_features], return_sequences=True))
model2.add(LSTM(128))
model2.add(Dropout(0.3))
model2.add(Dense(64, activation="relu"))
model2.add(Dropout(0.5))
model2.add(Dense(n_outputs, activation = "softmax"))

model2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 40, 64)            18176     
                                                                 
 lstm_7 (LSTM)               (None, 128)               98816     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 7)                 455       
                                                                 
Total params: 125703 (491.03 KB)
Trainable params: 125

In [168]:
model2.compile(optimizer=Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model2.fit(X_train, y_train, epochs = 40, batch_size = 32, validation_split = 0.1, shuffle = True)
print(model2.summary())

Epoch 1/40
15/15 [==============================] - 15s 278ms/step - loss: 1.8429 - accuracy: 0.3622 - val_loss: 1.7007 - val_accuracy: 0.3529
Epoch 2/40
15/15 [==============================] - 2s 110ms/step - loss: 1.4955 - accuracy: 0.4622 - val_loss: 1.5759 - val_accuracy: 0.3529
Epoch 3/40
15/15 [==============================] - 2s 103ms/step - loss: 1.4212 - accuracy: 0.4422 - val_loss: 1.5771 - val_accuracy: 0.3529
Epoch 4/40
15/15 [==============================] - 2s 103ms/step - loss: 1.3993 - accuracy: 0.4422 - val_loss: 1.5723 - val_accuracy: 0.2549
Epoch 5/40
15/15 [==============================] - 2s 106ms/step - loss: 1.4010 - accuracy: 0.4444 - val_loss: 1.6729 - val_accuracy: 0.2157
Epoch 6/40
15/15 [==============================] - 2s 103ms/step - loss: 1.3286 - accuracy: 0.4689 - val_loss: 1.5414 - val_accuracy: 0.3529
Epoch 7/40
15/15 [==============================] - 2s 113ms/step - loss: 1.3429 - accuracy: 0.4556 - val_loss: 1.5153 - val_accuracy: 0.2941
Epoch

In [174]:
base_loss,base_accuracy=model2.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)
modelpth2='pamap2.pth2'
tf.keras.models.save_model(model2, modelpth2, include_optimizer=False) 
score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))


300/300 [==============================] - 12s 39ms/step - loss: 2.0117 - accuracy: 0.3645
INFO:tensorflow:Assets written to: pamap2.pth2\assets


INFO:tensorflow:Assets written to: pamap2.pth2\assets


Accuracy >36.448988
Base Loss >2.01


CHEST - CHEST

In [175]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X, 
                                                    scaled_X["label"],
                                                    test_size = 0.95, 
                                                    shuffle = True, 
                                                    random_state = 21)

In [176]:
X_train, y_train = create_dataset(X_train[['chestAcc16_1', 'chestAcc16_2',  'chestAcc16_3',   
    'chestAcc6_1',  'chestAcc6_2',  'chestAcc6_3' ]], X_train.label, TIME_STEPS,
                                  STEP)
X_test, y_test = create_dataset(X_test[[
    'handAcc16_1', 'handAcc16_2',  'handAcc16_3',   
    'handAcc6_1',  'handAcc6_2',  'handAcc6_3']], X_test.label, TIME_STEPS, STEP)

print(X_train.shape, y_train.shape)

C:\Users\qtx01\AppData\Local\Temp\ipykernel_28264\302130513.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(stats.mode(labels)[0][0])
C:\Users\qtx01\AppData\Local\Temp\ipykernel_28264\302130513.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.a

(501, 40, 6) (501, 1)


In [177]:
enc = OneHotEncoder(handle_unknown = "ignore", sparse = False)
enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

C:\Users\qtx01\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [178]:
base_model2 = keras.models.load_model(modelpth2)

In [179]:
print("Number of layers in the base model: ", len(base_model2.layers))
fine_tune_at = 3

for layer in base_model2.layers[:fine_tune_at]:
    layer.trainable = False

new_model2 = Sequential(base_model2.layers[:-1])
new_model2.add(Dense(4, activation="sigmoid"))
new_model2.add(Dense(n_outputs, activation="sigmoid"))
new_model2.compile(loss="categorical_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])
new_model2.summary()

Number of layers in the base model:  6


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 40, 64)            18176     
                                                                 
 lstm_7 (LSTM)               (None, 128)               98816     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 4)                 260       
                                                                 
 dense_11 (Dense)            (None, 7)                

In [180]:
display_layers = [(layer, layer.name, layer.trainable) for layer in base_model2.layers]
pd.DataFrame(display_layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.src.layers.rnn.lstm.LSTM object at 0x00...,lstm_6,False
1,<keras.src.layers.rnn.lstm.LSTM object at 0x00...,lstm_7,False
2,<keras.src.layers.regularization.dropout.Dropo...,dropout_6,False
3,<keras.src.layers.core.dense.Dense object at 0...,dense_8,True
4,<keras.src.layers.regularization.dropout.Dropo...,dropout_7,True
5,<keras.src.layers.core.dense.Dense object at 0...,dense_9,True


In [181]:
history3 = new_model2.fit(X_train, y_train, epochs=10, batch_size = 32, validation_split = 0.1, shuffle = True)
new_model2.summary()

Epoch 1/10
15/15 [==============================] - 12s 224ms/step - loss: 2.2768 - accuracy: 0.0111 - val_loss: 2.1427 - val_accuracy: 0.0196
Epoch 2/10
15/15 [==============================] - 1s 58ms/step - loss: 2.2047 - accuracy: 0.0044 - val_loss: 2.0862 - val_accuracy: 0.0196
Epoch 3/10
15/15 [==============================] - 1s 57ms/step - loss: 2.1237 - accuracy: 0.0111 - val_loss: 2.0363 - val_accuracy: 0.0196
Epoch 4/10
15/15 [==============================] - 1s 53ms/step - loss: 2.0631 - accuracy: 0.0378 - val_loss: 1.9907 - val_accuracy: 0.0392
Epoch 5/10
15/15 [==============================] - 1s 59ms/step - loss: 1.9980 - accuracy: 0.0800 - val_loss: 1.9511 - val_accuracy: 0.0588
Epoch 6/10
15/15 [==============================] - 1s 57ms/step - loss: 1.9460 - accuracy: 0.1400 - val_loss: 1.9156 - val_accuracy: 0.2549
Epoch 7/10
15/15 [==============================] - 1s 56ms/step - loss: 1.8978 - accuracy: 0.2933 - val_loss: 1.8874 - val_accuracy: 0.3333
Epoch 8/10


In [182]:
base_model2.trainable = True
new_model2.compile(loss="categorical_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [183]:

display_layers = [(layer, layer.name, layer.trainable) for layer in base_model2.layers]
pd.DataFrame(display_layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.src.layers.rnn.lstm.LSTM object at 0x00...,lstm_6,True
1,<keras.src.layers.rnn.lstm.LSTM object at 0x00...,lstm_7,True
2,<keras.src.layers.regularization.dropout.Dropo...,dropout_6,True
3,<keras.src.layers.core.dense.Dense object at 0...,dense_8,True
4,<keras.src.layers.regularization.dropout.Dropo...,dropout_7,True
5,<keras.src.layers.core.dense.Dense object at 0...,dense_9,True


In [184]:
history4= new_model2.fit(X_train, y_train, epochs=10, batch_size = 32, validation_split = 0.1, shuffle = True)
new_model2.summary()

Epoch 1/10
15/15 [==============================] - 14s 293ms/step - loss: 1.7526 - accuracy: 0.4867 - val_loss: 1.7904 - val_accuracy: 0.3529
Epoch 2/10
15/15 [==============================] - 2s 118ms/step - loss: 1.7107 - accuracy: 0.5244 - val_loss: 1.7698 - val_accuracy: 0.3529
Epoch 3/10
15/15 [==============================] - 2s 128ms/step - loss: 1.6919 - accuracy: 0.5000 - val_loss: 1.7540 - val_accuracy: 0.3529
Epoch 4/10
15/15 [==============================] - 2s 121ms/step - loss: 1.6590 - accuracy: 0.4800 - val_loss: 1.7412 - val_accuracy: 0.3529
Epoch 5/10
15/15 [==============================] - 2s 120ms/step - loss: 1.6424 - accuracy: 0.4756 - val_loss: 1.7284 - val_accuracy: 0.3529
Epoch 6/10
15/15 [==============================] - 2s 116ms/step - loss: 1.6144 - accuracy: 0.4711 - val_loss: 1.7133 - val_accuracy: 0.3529
Epoch 7/10
15/15 [==============================] - 2s 129ms/step - loss: 1.5976 - accuracy: 0.4711 - val_loss: 1.7018 - val_accuracy: 0.3529
Epoch

In [185]:
base_loss,base_accuracy=new_model2.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)
  
score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))

300/300 [==============================] - 14s 45ms/step - loss: 1.6421 - accuracy: 0.3843
Accuracy >38.431045
Base Loss >1.64


CHEST - HAND